In [ ]:
import riiideducation
import dask.dataframe as dd
import  pandas as pd
import numpy as np
env = riiideducation.make_env()

In [ ]:
train= pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                usecols=[1, 2, 3,7], dtype={'timestamp': 'int64', 'user_id': 'int64' ,'content_id': 'int16','answered_correctly':'int8'}
              )

In [ ]:
train.head()

In [ ]:
#Make a simple logistic model and estimate score

In [ ]:
#arrange by timestamp

train = train.sort_values(['timestamp'], ascending=True)
train.head()

In [ ]:
train.tail()

In [ ]:
train.drop(['timestamp'], axis=1,   inplace=True)

In [ ]:
len(list(set(train.iloc[0:90000000,:]['user_id']).intersection(set(train.iloc[90000000:99000000,:]['user_id']))))

In [ ]:
results_c = train.iloc[0:90000000,:][['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

In [ ]:
results_u = train.iloc[0:90000000,:][['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u.columns = ["answered_correctly_user"]

In [ ]:
X = train.iloc[90000000:99000000,:]


In [ ]:
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")

In [ ]:
X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])


In [ ]:
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)
X.head()

In [ ]:
Y

In [ ]:
X = X[['answered_correctly_user', 'answered_correctly_content']] 
X.fillna(0.5,  inplace=True)

In [ ]:
from  sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size =0.2, shuffle=False)

Reg = LogisticRegression()
Reg.fit(Xt, np.array(Yt))

In [ ]:
y_pred = Reg.predict_proba(Xv)[:,1]
#y_pred = np.array(Xv['answered_correctly_content'])
y_true = np.array(Yv)

In [ ]:
Y.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred)

In [ ]:
###Make sure it works on the test set

In [ ]:
test =  pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
test.shape

In [ ]:
test = pd.merge(test, results_u, on=['user_id'],  how="left")
test = pd.merge(test, results_c, on=['content_id'],  how="left")

In [ ]:
test.shape

In [ ]:
test[['answered_correctly_user', 'answered_correctly_content']]
test.fillna(0.5, inplace=True)

In [ ]:
y_pred = Reg.predict_proba(test[['answered_correctly_user', 'answered_correctly_content']])[:,1]

In [ ]:
y_pred.shape

In [ ]:
test['answered_correctly'] = Reg.predict_proba(test[['answered_correctly_user', 'answered_correctly_content']])[:,1]
test.head()

In [ ]:
#################

In [ ]:
#Make preds

results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u.columns = ["answered_correctly_user"]

In [ ]:
results_c.head()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = pd.merge(test_df, results_u, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_c, on=['content_id'],  how="left")
    test_df['answered_correctly_user'].fillna(0.5, inplace=True)
    test_df['answered_correctly_content'].fillna(0.5, inplace=True)
    test_df['answered_correctly'] =  Reg.predict_proba(test_df[['answered_correctly_user', 'answered_correctly_content']])[:,1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])